## this is a draft to follow the performance of several mice across sessions for a set of behavioral parameters saved in pickled files 

In [2]:
import os
import pickle
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt #Creation de graphiques et de visualisations comme matlab
from matplotlib.patches import Polygon #Creation de graphiques et de visualisations comme matlab
from matplotlib import cm
import matplotlib.patches as mpatches
import gc
import glob
import time
from behavior_analysis_functions import list_sessions_analyzed, \
                                    process_session, html_mouse, load_data, MOUSE_STRING


In [32]:
# the path_to_data_folder is the path of the folder where you store your different mice.

# Windows:
# path_to_data_folder='C:'+os.sep+'Users'+os.sep+'MORVAN'+os.sep+'Documents'+os.sep+'patchouris'+os.sep+'Sample_Data'+os.sep
# Linux:
# path_to_data_folder '/home/david/Documents/Code/ForagingProject/Patchouris/patchouris/Sample_Data/'
# MacOs
# path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'

path_to_data_folder='/home/david/Documents/David/Data/'

#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'

# List all mice in the data folder (If you want to process all the mice in your data folder),
mice_list = [os.path.basename(path) for path in glob.glob(path_to_data_folder + "/MOU*")]
mice_list=sorted(mice_list)
print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')


#If you want to process a subset of mice uncomment the line below and comment the 2 lines above

#mice_list: list[str] = ["MOU2334"]  # For processing  a single mice. Equivalent to mice_list = ["MOU2334"] but more correct as it forces to create a list of string
#print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')

session_list = {}
for mouse in mice_list:
    mouse_folder = path_to_data_folder + mouse
    session_list[mouse] = [name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')]
    nb_sessions = len(session_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(session_list[mouse], '\n')






Found 6 mice in the data folder:
Hello, I'm MOU2329! I have foraged for 22 sessions.
['MOU2329_20240613-0914', 'MOU2329_20240610-1336', 'MOU2329_20240613-1405', 'MOU2329_20240617-0829', 'MOU2329_20240619-0828', 'MOU2329_20240612-0918', 'MOU2329_20240618-1451', 'MOU2329_20240618-0829', 'MOU2329_20240621-0829', 'MOU2329_20240614-0820', 'MOU2329_20240620-1339', 'MOU2329_20240617-1420', 'MOU2329_20240620-0821', 'MOU2329_20240611-0915', 'MOU2329_20240610-0936', 'MOU2329_20240607-1315', 'MOU2329_20240607-1714', 'MOU2329_20240619-1406', 'MOU2329_20240611-1324', 'MOU2329_20240614-1331', 'MOU2329_20240621-1425', 'MOU2329_20240612-1341'] 

Hello, I'm MOU2330! I have foraged for 22 sessions.
['MOU2330_20240617-0848', 'MOU2330_20240613-0933', 'MOU2330_20240617-1438', 'MOU2330_20240620-1357', 'MOU2330_20240619-0846', 'MOU2330_20240621-0848', 'MOU2330_20240619-1424', 'MOU2330_20240620-0838', 'MOU2330_20240607-1733', 'MOU2330_20240614-0840', 'MOU2330_20240611-0936', 'MOU2330_20240612-1400', 'MOU2330_

"""
### Description of `load_pickle_data` Function

The `load_pickle_data` function is designed to recursively traverse a directory structure containing data organized by mice and sessions. It loads variables stored in pickle files across all sessions for a given list of mice.

#### Function Flow:

1. **Initialization**:
   - Initialize an empty dictionary `data` to store loaded data.

2. **Loop Over Mice**:
   - For each mouse specified in `mice_list`, construct the path to the mouse's data folder (`mouse_folder`).
   - Check if `mouse_folder` exists; if not, print a message and skip to the next mouse.

3. **List Session Folders**:
   - Retrieve a list of session folders (`session_folders`) within `mouse_folder`.
   - Filter `session_folders` to include only folders starting with "MOU" using a list comprehension.

4. **Iterate Over Session Folders**:
   - For each session folder (`session_folder`) in `session_folders`, construct the path to `session_folder`.
   - Check if a `Pickle_data` folder exists within `session_folder`.
   - If `Pickle_data` folder is found, initialize an empty dictionary under `data[mouse][session_folder]` to store loaded variables.

5. **Load Pickle Files**:
   - Iterate over `pickle_files` list to specify which pickle files to load.
   - Construct the path to each pickle file (`pickle_file_path`) within `Pickle_data` folder.
   - Check if `pickle_file_path` exists as a file; if not, print a message and skip to the next file.
   - Open and load the pickle file (`pickle.load(pf)`) into `loaded_data`.
   - Use the base name of the pickle file (without extension) as the key in `data[mouse][session_folder]` to store `loaded_data`.

6. **Error Handling**:
   - Catch any exceptions that occur during the loading of pickle files and print an error message.

7. **Return Data**:
   - Return the populated `data` dictionary containing loaded variables from pickle files across all specified mice and sessions.



In [30]:
import os
import pickle
import numpy as np

def load_pickle_data(path_to_data_folder, mice_list, pickle_files):
    data = {}

    for mouse in mice_list:
        mouse_folder = os.path.join(path_to_data_folder, mouse)
        if not os.path.isdir(mouse_folder):
            print(f"Mouse folder not found: {mouse_folder}")
            continue
        
        data[mouse] = {}

        # List session folders starting with 'MOU' within the mouse folder
        session_folders = [folder for folder in os.listdir(mouse_folder) 
                           if folder.startswith('MOU') and os.path.isdir(os.path.join(mouse_folder, folder))]

        for session_folder in session_folders:
            session_folder_path = os.path.join(mouse_folder, session_folder)
            pickle_data_folder = os.path.join(session_folder_path, 'Pickle_data')

            if not os.path.isdir(pickle_data_folder):
                print(f"Pickle_data folder not found in {session_folder_path}")
                continue
            
            data[mouse][session_folder] = {}

            # Load data from pickle files within Pickle_data folder
            for pickle_file in pickle_files:
                pickle_file_path = os.path.join(pickle_data_folder, pickle_file)
                
                if not os.path.isfile(pickle_file_path):
                    print(f"Pickle file not found: {pickle_file_path}")
                    continue
                
                try:
                    with open(pickle_file_path, 'rb') as pf:
                        loaded_data = pickle.load(pf)
                        # Use the pickle file name as key instead of session folder name
                        pickle_file_name = os.path.splitext(pickle_file)[0]  # Extract file name without extension
                        data[mouse][session_folder][pickle_file_name] = loaded_data
                except Exception as e:
                    print(f"Error loading pickle file {pickle_file_path}: {e}")

    return data

# # Example usage
# path_to_data_folder = '/home/david/Documents/David/Data/'
# mice_list = ['MOU2334']  # List of mice names
# pickle_files = ['distance_traveled.pkl']  # List of pickle files to load
# data = load_pickle_data(path_to_data_folder, mice_list, pickle_files)

# # Accessing the loaded data
# print(data['MOU2334']['MOU2334_20240607-1213'])  # Replace with actual session folder name


In [34]:
pickle_files = ['distance_traveled.pkl']  # List of pickle files to load
data = load_pickle_data(path_to_data_folder, mice_list, pickle_files)

In [35]:
data

{'MOU2329': {'MOU2329_20240613-0914': {'distance_traveled': np.float64(91.61292345296579)},
  'MOU2329_20240610-1336': {'distance_traveled': np.float64(60.913424162525025)},
  'MOU2329_20240613-1405': {'distance_traveled': np.float64(95.90301630181803)},
  'MOU2329_20240617-0829': {'distance_traveled': np.float64(100.0676237661088)},
  'MOU2329_20240619-0828': {'distance_traveled': np.float64(138.8581889117273)},
  'MOU2329_20240612-0918': {'distance_traveled': np.float64(96.30183951935192)},
  'MOU2329_20240618-1451': {'distance_traveled': np.float64(118.82672243105695)},
  'MOU2329_20240618-0829': {'distance_traveled': np.float64(101.67168904370914)},
  'MOU2329_20240621-0829': {'distance_traveled': np.float64(125.48198014705929)},
  'MOU2329_20240614-0820': {'distance_traveled': np.float64(102.73465236340373)},
  'MOU2329_20240620-1339': {'distance_traveled': np.float64(102.86873674801406)},
  'MOU2329_20240617-1420': {'distance_traveled': np.float64(104.47763250810011)},
  'MOU2329

In [36]:
pickle_files = ['distance_traveled.pkl','consecutive_quarter_turns.pkl']  # List of pickle files to load
data = load_pickle_data(path_to_data_folder, mice_list, pickle_files)

In [37]:
data

{'MOU2329': {'MOU2329_20240613-0914': {'distance_traveled': np.float64(91.61292345296579),
   'consecutive_quarter_turns': [[np.float64(25.405),
     np.float64(36.05),
     'SW',
     5],
    [np.float64(35.289), np.float64(56.941), 'NW', 9],
    [np.float64(56.179), np.float64(67.948), 'NE', 5],
    [np.float64(67.305), np.float64(73.63), 'SE', 4],
    [np.float64(72.95), np.float64(77.672), 'SW', 3],
    [np.float64(76.992), np.float64(85.316), 'NW', 5],
    [np.float64(84.637), np.float64(92.281), 'NE', 4],
    [np.float64(91.72), np.float64(98.362), 'SW', 4],
    [np.float64(97.563), np.float64(107.002), 'SE', 5],
    [np.float64(105.769), np.float64(109.289), 'SW', 1],
    [np.float64(108.725), np.float64(115.453), 'NW', 4],
    [np.float64(114.85), np.float64(121.095), 'SW', 3],
    [np.float64(120.453), np.float64(130.578), 'NE', 8],
    [np.float64(129.98), np.float64(137.583), 'SE', 4],
    [np.float64(136.983), np.float64(151.633), 'NW', 4],
    [np.float64(150.991), np.floa